# Ex_moviプロジェクト Tips 集

MOVi-A データセット探索・SAVi-DINOSAUR 実装を通じて得られた知見をまとめる。

---
## 1. 環境・容量関連

### `.venv` が 10GB になるのは普通か？

**普通である。** 機械学習環境では以下が大きい：

| パッケージ | 概算サイズ |
|-----------|----------|
| PyTorch (CUDA版) | 2〜3 GB |
| TensorFlow | 1〜2 GB |
| CUDA関連ライブラリ (nvidia/) | 2〜4 GB |
| NumPy/SciPy/Pandas等 | 数百MB |

確認コマンド：
```bash
du -h --max-depth=2 .venv/lib/python3.10/site-packages | sort -rh | head -15
```

---
## 2. データ形式関連

### `.pt` ファイルとは

**PyTorch の標準シリアライズ形式**である。

```python
# 保存
torch.save(data, "sample.pt")

# 読み込み
data = torch.load("sample.pt", weights_only=False)
```

| 項目 | 説明 |
|------|------|
| 拡張子 | `.pt` または `.pth`（どちらも同じ） |
| 形式 | Python pickle + PyTorch独自最適化 |
| 保存可能 | テンソル、dict、list、モデル重みなど |
| 利点 | 高速な読み書き、GPU/CPUテンソル対応 |

### TensorFlow → PyTorch 変換で容量が増える理由

| 項目 | TensorFlow (変換前) | PyTorch (変換後) | 倍率 |
|------|---------------------|------------------|------|
| dtype | **uint8** (1バイト) | **float32** (4バイト) | **4倍** |
| 値の範囲 | 0-255 | 0.0-1.0 | - |

**理由**: PyTorchのニューラルネットワークは通常 `float32` の入力を期待するため。

```python
# 変換コード
video = video.float() / 255.0  # uint8 → float32
```

**容量を減らす選択肢**（必要な場合のみ）：
- `uint8` で保存 → 学習時に毎回 `/ 255.0` が必要
- `float16` で保存 → 精度がわずかに落ちる可能性

### PyTorch のテンソル軸の順序

| フレームワーク | 画像の軸順序 | 動画の軸順序 |
|--------------|-------------|-------------|
| TensorFlow | H, W, C (channels last) | T, H, W, C |
| PyTorch | C, H, W (channels first) | T, C, H, W |

```python
# TensorFlow → PyTorch 変換
video = video.permute(0, 3, 1, 2)  # THWC → TCHW
```

---
## 3. データセット関連

### MOVi-A で TensorFlow を使う理由

MOVi-A は **Google Kubric プロジェクト**が提供しており、公式配布形式が **TFRecord（TensorFlow Datasets）のみ**である。

- GCS上の `gs://kubric-public/tfds` から直接読み込む場合、`tensorflow-datasets` が必須
- ただし、読み込み後は即座に **PyTorchテンソルに変換**して保存すれば、学習時にはTensorFlow不要

```python
# 読み込み（TensorFlow必須）
ds = tfds.load("movi_a/256x256", data_dir="gs://kubric-public/tfds")

# 変換して保存（以降TensorFlow不要）
torch.save(pytorch_sample, "sample.pt")
```

### GCS ストリーミング vs ローカル保存

| 方式 | メリット | デメリット |
|------|---------|----------|
| GCSストリーミング | ローカル容量不要 | 毎回ネットワーク通信、遅い |
| ローカル保存 (.pt) | 高速、オフライン可 | ディスク容量必要 |

**推奨**: 小規模サブセットを `.pt` で保存し、学習・評価に使用する。

---
## 4. Jupyter Notebook 関連

### 動画をノートブック内で再生する方法

2つの方法がある：

| 方式 | `to_jshtml()` | `to_html5_video()` |
|------|---------------|-------------------|
| 依存 | なし | **ffmpeg必須** |
| ファイルサイズ | 大きめ（各フレームPNG） | 小さい（H.264圧縮） |
| 再生 | JavaScript制御 | ブラウザネイティブ |

```python
from matplotlib import animation
from IPython.display import HTML

anim = animation.FuncAnimation(fig, update, frames=24)

# ffmpegなしで動画表示
HTML(anim.to_jshtml())

# ffmpegありならこちらの方が軽量
# HTML(anim.to_html5_video())
```

**注意**: `ffmpeg` は Python パッケージではなくシステムバイナリ。
```bash
# uv/pip ではインストール不可
sudo apt install ffmpeg  # Ubuntu/Debian
```

### ノートブックの容量について

- **コード・マークダウン**: 数十KB〜数百KB
- **画像出力**: 各画像数十KB〜数百KB
- **動画（jshtml）**: 数MB程度

**GBレベルにはならない**。出力をクリアして保存すればさらに軽量。

---
## 5. Git 関連

### ローカルがリモートより遅れている場合

```bash
# 状況確認
git fetch origin
git status

# 最新を取得（ローカル変更なし）
git pull --ff-only

# ローカル変更あり
git stash
git pull --rebase
git stash pop
```

---
## 6. パッケージ管理 (uv)

### uv でのパッケージインストール

```bash
# パッケージインストール
uv pip install torch tensorflow-datasets

# requirements.txt から一括インストール
uv pip install -r requirements.txt

# 容量確認
du -sh .venv
```

**注意**: `ffmpeg` 等のシステムバイナリは `uv` ではインストールできない。

---
## 7. MOVi-A データセット固有の知見

### ラベルは整数インデックス

MOVi-A のラベルは文字列ではなく**整数**で格納されている：

```python
SHAPE_LABELS = ["cube", "cylinder", "sphere"]      # 0, 1, 2
MATERIAL_LABELS = ["metal", "rubber"]              # 0, 1
COLOR_LABELS = ["blue", "brown", "cyan", "gray", "green", "purple", "red", "yellow"]
SIZE_LABELS = ["small", "large"]                   # 0, 1
```

### 材質分布の偏り

100サンプル調査結果：
- Metal only: **6%**
- Rubber only: **0%** ← 純粋なゴムのみのシーンは存在しない
- Mixed: **94%**

**含意**: 比較実験は「金属のみ」vs「混合」で行う必要がある。

---
## 8. デバッグ Tips

### テンソルの中身を確認

```python
print(f"形状: {tensor.shape}")
print(f"dtype: {tensor.dtype}")
print(f"値の範囲: [{tensor.min():.3f}, {tensor.max():.3f}]")
print(f"メモリ: {tensor.numel() * tensor.element_size() / 1024 / 1024:.2f} MB")
```

### .pt ファイルの構造確認

```python
sample = torch.load("file.pt", weights_only=False)
for key, value in sample.items():
    print(f"{key}: {type(value).__name__}")
```